In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
print(torch.cuda.is_available())

True


In [3]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1) #256, 128, 64, 32, 16
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)                               #16x16x32
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)#16x16x64
#         self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)                            #8x8x64
        self.fc1 = nn.Linear(32*8*8, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.activation = nn.GELU()

    def forward(self, x):
        x = self.activation(self.conv1(x)) #256x256
        x = self.pool(x)                   #128x128
        x = self.activation(self.conv2(x)) 
        x = self.pool(x)                   #64x64
        x = self.activation(self.conv2(x))
        x = self.pool(x)                   #32x32
        x = self.activation(self.conv2(x))
        x = self.pool(x)                   #16x16
        x = self.activation(self.conv2(x))
        x = self.pool(x)                   #8x8
        x = x.view(-1, 8*8*32)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# Hyperparameters
num_classes = 10
learning_rate = 0.003
batch_size = 100
num_epochs = 10

# Load and transform the data
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])



train_dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/train', transform=transform)

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [8000, 1999])

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/val', transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Initialize the CNN
model = CNN(num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def accuracy(self, loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print('Accuracy: ', accuracy)
        
# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    total_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        if (i+1)%10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Avg Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, total_loss/(i+1)))

    accuracy(val_loader)
    print("\n")
    
accuracy(test_loader)
# Optionally, save the trained model
torch.save(model.state_dict(), 'cnn_model.ckpt')

Epoch [1/10], Step [10/80], Avg Loss: 2.3244
